In [62]:
import numpy as np
import pandas as pd
import requests
from api_keys import iex_key, fmp_key
import time
import re

In [63]:
def get_annual_dividend(stock_symbol):
    # Make request to fmp API.
    fmp_base_url = 'https://fmpcloud.io/api/v3/profile/'
    params = {'apikey': fmp_key}
    req = requests.get(fmp_base_url + stock_symbol, params = params)
    div_list = req.json()
    
    # If API request returns and empty JSON, return -1. Else, return the lastDiv value of the first result.
    if not div_list:
        return -1.00
    else:
        return float(div_list[0]['lastDiv'])

In [64]:
def populate_annual_dividends(df):
    # Get a list of all stock symbols.
    symbols = df.symbol.values.tolist()
    
    annual_dividends = []
    
    # Loop through symbols to obtain dividend data, sleeping to respect API call limits.
    for ix, symb in enumerate(symbols):
        if ix % 300 == 0 and ix != 0:
            time.sleep(65)
            
        annual_dividends.append(get_annual_dividend(symb))
    
    return annual_dividends

In [65]:
def get_symbols_and_dividends():
    # Grab symbols list using iex key.
    # Split by formatting and select only the column containing stock symbols and convert to Pandas DataFrame.
    iex_base_url = 'https://api.iex.cloud/v1/tops/last'
    params = {'format': 'csv', 'token': iex_key}
    symbol_csv = requests.get(iex_base_url, params = params).text
    symbol_csv_list = symbol_csv.split('\r\n')
    df = pd.DataFrame([x.split(',')[0] for x in symbol_csv_list][1:], columns = ['symbol'])
    
    # Get dividend values.
    annual_dividends = populate_annual_dividends(df)
    
    # Turn into DataFrame and merge with symbols.
    dividends_df = pd.DataFrame(annual_dividends, columns = 'lastDiv')
    merged_df = pd.concat([df, dividends_df], axis = 1)
    
    return merged_df

In [43]:
df = get_symbols_and_dividends()
pd.to_csv(df, 'dividends.csv')